In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Activation, BatchNormalization, Concatenate, Dense, Embedding, Flatten, Reshape, Input, Multiply
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
from tensorflow.python.keras.layers.convolutional import Conv1DTranspose

In [3]:
from gensim.models import word2vec
import MeCab
import json
import hashlib
from googletrans import Translator
from keras.layers import Dropout
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import torch
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
%matplotlib inline

In [4]:
translator = Translator()
mt = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
mt.parse('')
model_word = word2vec.Word2Vec.load("wiki_plus.model")

In [5]:
def get_tags(text):
    word = {}
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if (fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞') and node.surface in model_word.wv:
            w = node.surface
            word[w] = word.get(w, 0) + 1
        node = node.next
    return word

def weighted_mean_vec(text):
    v = np.zeros(model_word.vector_size)
    s = 1.0
    for w,weight in get_tags(text).items():
        v += weight * model_word.wv[w]  #Eventクラスeの単語wの個数＊単語wのベクトル
        s += weight
    return v / s

In [6]:
class Event:
    def __init__(self, id, type, score, desc, links):
        self.id = id
        self.type = type
        self.score = score
        self.desc = desc
        self.links = links

In [7]:
# JSON ファイルから event set をロード
def load_events(jsonfile):
    with open(jsonfile) as f:
        df = json.load(f)
    events = {x['id']: Event(x['id'], x['type'], x['score'], x['desc'], x['links']) for x in df} #eventsにidをkeyとしそのオブジェクトをvalueとした辞書を生成
    for k,x in events.items():
        x.links = [events[e] for e in x.links] #Event.linkの中身をidの配列からEventの配列に変更
    return events

In [8]:
events = load_events('sesaku2.json')

In [9]:
import pandas as pd
data = []
labels = []
columns=[]
index=[]
for k1, v1 in events.items():
    if v1.type[-1]=='部品':
        index.append(v1.desc)
for k1, v1 in events.items():
    if v1.type[-1]=='対策':
        if not v1.desc in columns:
            columns.append(v1.desc)
df = pd.DataFrame(index=index, columns=columns)
for k1, v1 in events.items():
    if v1.type[-1]=='部品':
        for k2, v2 in events.items():
            if v2.type[-1] == '対策':
                    if v2 in v1.links:
                        df.at[v1.desc, v2.desc] = 1
                    else:
                        df.at[v1.desc, v2.desc] = 0

In [10]:
num_classes = 173

In [11]:
taisaku_vec_word = {}
for i in df:
    taisaku_vec_word[i]=weighted_mean_vec(i)

#部品ごとにラベル付け
buhin_labels = {}
labels_buhin = {}
i = 0
for index, row in df.iterrows():
    buhin_labels[index] = i
    labels_buhin[i] = index
    i +=1
    

#label →部品　生成→対策
data_word = []
labels_word = []
# data 
for index, row in df.iterrows():#部品
    x1 = weighted_mean_vec(index)
    x1 = buhin_labels[index]
    for i in df:#対策
        x2 =  taisaku_vec_word[i]#対策
        if row[i] ==1:
            data_word.append(x2)
            labels_word.append(x1)

In [12]:
data_max = np.array(data_word).max()
data_max

3.9517772674560545

In [13]:
data_min= np.array(data_word).min()
data_min

-4.16222607344389

In [14]:
 #生成器
def build_generator(z_dim):
    
        model = Sequential()
        
        model.add(Dense(50 *256, input_dim = z_dim))
        
        model.add(Reshape((50,256)))
        
        #転置畳み込み層により,50×256を100×128のテンソルに変換
        model.add(Conv1DTranspose(128, kernel_size=3, strides=2, padding='same'))
        
        #バッチ正規化
        model.add(BatchNormalization())
        
        model.add(LeakyReLU(alpha=0.01))
        
        #転置畳み込み層により,100×16を100×8のテンソルに変換
        model.add(Conv1DTranspose(64, kernel_size=3, strides=1, padding='same'))
        
        model.add(BatchNormalization())
        
        model.add(LeakyReLU(alpha=0.01))
        
        #転置畳み込み層により,100×64を200×1をのテンソルに変換
        model.add(Conv1DTranspose(1, kernel_size=3, strides=2, padding='same'))
        
        model.add(Activation('tanh'))

        return model

In [15]:
def bulid_cgan_generator(z_dim):
    
    z = Input(shape=(z_dim, ))
    
    label = Input(shape=(1, ), dtype='int32')
    
    label_embedding = Embedding(num_classes, z_dim, input_length=1)(label)
    
    label_embedding = Flatten()(label_embedding)
    
    joined_representation = Multiply()([z, label_embedding])
    
    generator = build_generator(z_dim)
    
    conditioned_vec = generator(joined_representation)
    
    return Model([z, label], conditioned_vec)

In [16]:
#識別器

def build_discriminator(vec_shape):
    
    model = Sequential()

    #200×2を100×64のテンソルにする畳み込み層

    model.add(Conv1D(64, kernel_size=3, strides=2,input_shape=(vec_shape[0],vec_shape[1]+1), padding='same'))
    
    model.add(LeakyReLU(alpha=0.01))
    
    #100×64を50×64のテンソルにする畳み込み層
    model.add(Conv1D(64, kernel_size=3, strides=2, input_shape=vec_shape, padding='same'))
    
    #バッチ正規化
    model.add(BatchNormalization())
    
    model.add(LeakyReLU(alpha=0.01))
    
    #50×64を25×128のテンソルにする畳み込み層
    
    model.add(Conv1D(128, kernel_size=3, strides=2, input_shape=vec_shape, padding='same'))
    
    #バッチ正規化
    model.add(BatchNormalization())
    
    model.add(LeakyReLU(alpha=0.01))
    
    model.add(Flatten())
        
    #出力にシグモイド関数を適用
    model.add(Dense(1, activation='sigmoid'))
    return model

In [17]:
def build_cgan_discriminator(vec_shape):
    
    vec = Input(shape=vec_shape)
    
    label = Input(shape=(1, ),dtype='int32')
    
    label_embedding = Embedding(num_classes, np.prod(vec_shape), input_length=1)(label)
    
    label_embedding = Flatten()(label_embedding)
    
    label_embedding = Reshape(vec_shape)(label_embedding)
    
    concatenated = Concatenate(axis=-1)([vec, label_embedding])
    
    discriminator = build_discriminator(vec_shape)
    
    classification = discriminator(concatenated)
    
    return Model([vec, label], classification)


def build_cgan(generator, discriminator):
    
    z = Input(shape=(z_dim, ))
    
    label = Input(shape=(1,))
    
    vec = generator([z, label])
    
    classification = discriminator([vec, label])
    
    model = Model([z, label], classification)
    
    return model



def build_compile(data, z_dim):
    #モデルの入力次元
    data = np.array(data)
    data = np.reshape(data, (-1, data.shape[1], 1))
    vec_shape = data.shape[1:]
    #識別器の構築とコンパイル
    discriminator = build_cgan_discriminator(vec_shape)
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    #生成器の構築
    generator = bulid_cgan_generator(z_dim)

    #生成器の構築中は識別器のパラメータを固定
    discriminator.trainable = False

    #生成器の訓練のため、識別器は固定し、GANモデルの構築とコンパイルを行う
    cgan = build_cgan(generator, discriminator)
    cgan.compile(loss='binary_crossentropy', optimizer=Adam())
    
    return discriminator, generator, cgan

In [18]:
losses = []
accuracies = []


def train(data, label, iterations, batch_size, sample_interval):
    
    #1の数7303
    data = np.array(data)
    
    #-1~1にスケーリング
    data = (data - data.min()) / (data.max() -data.min())
    data = data*2-1
    
    data = np.reshape(data, (-1, data.shape[1], 1))
    label = np.array(label)
    #ラベル1
    real = np.ones((batch_size,1))
    #ラベル0
    fake = np.zeros((batch_size,1))
    
    
    for iteration in range(iterations):
        
        #-------------------
        #識別器の訓練
        #-------------------
        
        #ランダムに関係があるベクトルをとる
        idx = np.random.randint(0, data.shape[0], batch_size)
        vecs, labels = data[idx], label[idx]
        
        z = np.random.normal(0, 1,(batch_size, z_dim))
        gen_vec = generator.predict([z,labels])
        
        d_loss_real = discriminator.train_on_batch([vecs,labels], real)
        d_loss_fake = discriminator.train_on_batch([gen_vec,labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        #-------------------
        #生成器の訓練
        #-------------------

        z = np.random.normal(0, 1,(batch_size, z_dim))
        
        labels = np.random.randint(0, num_classes, batch_size).reshape(-1,1)

        g_loss = cgan.train_on_batch([z, labels], real)
        
        if(iteration +1) % sample_interval == 0:
        
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %(iteration +1,d_loss[0], 100.0*d_loss[1],g_loss))
        
            losses.append((d_loss[0], g_loss))
            accuracies.append(100.0 * d_loss[1])

    return generator

In [19]:
z_dim = 100

discriminator, generator, cgan = build_compile(data_word, z_dim)

iterations =6000
batch_size = 32
sample_interval = 1000

generator = train(data_word, labels_word, iterations, batch_size,sample_interval)

1000 [D loss: 0.000222, acc.: 100.00%] [G loss: 4.572573]
2000 [D loss: 0.000132, acc.: 100.00%] [G loss: 0.022422]
3000 [D loss: 0.000030, acc.: 100.00%] [G loss: 0.003127]
4000 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.000447]
5000 [D loss: 0.000001, acc.: 100.00%] [G loss: 0.000446]
6000 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.001561]


In [20]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
class pycolor:
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    PURPLE = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    RETURN = '\033[07m' #反転
    ACCENT = '\033[01m' #強調
    FLASH = '\033[05m' #点滅
    RED_FLASH = '\033[05;41m' #赤背景+点滅
    END = '\033[0m'

In [27]:
data_max = np.array(data_word).max()
data_min= np.array(data_word).min()
def  test(label_number):
    print('##########部品##########')
    print(labels_buhin[label_number])
    
    print('##########対策##########')
    taisaku = []
    for index, row in df.iterrows():#部品
        if (index == labels_buhin[label_number] ):
            for i in df:#対策
                if row[i] ==1:
                    print(i)
                    taisaku.append(i)
            break
    z = np.random.normal(0,1,(3, z_dim))
    labels =np.full(3, label_number).reshape(-1,1)
    z_vec = generator.predict([z,labels])
    z_vec = z_vec.reshape(3,200)
    z_vec = 2*(z_vec-data_min)/(data_max - data_min) - 1
    #z_vec = z_vec*(data_max- data_min)/2+1*data_min
    print('##########出力##########')
    count = 0
    for i in range(3):
        score_sim = {}
        for k,v in taisaku_vec_word.items():
            score_sim[k]=cos_sim(z_vec[i], v)
        score_sort = sorted(score_sim.items(),key=lambda x:x[1], reverse=True)
        for i_1, i_2 in score_sort[0:10]:
            print(i_1)
            print(i_2)
            if i_1 in taisaku:
                print(pycolor.BLUE+"ある"+pycolor.END)
                count +=1
            else:
                print(pycolor.RED+"なし"+pycolor.END)
    print(count/30*100,'%')

In [28]:
test(1)

##########部品##########
エンジン搭載 燃料ポンプ 抵抗器アセンブリ
##########対策##########
多能工化を設計者に適用。メカ・電気・ソフトの各スキルマップを作成し統合。
定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。
６ヶ月に一回のサイクルで実施。
会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している
背景・目的・目標・KGI/KPI・大日程、現状の課題までが繋がっている
関連部門が参加した課題解決の仕組みがあり、PLMを使ったデジタル化によるグローバル会議ができる
機能と役割分担表があり、リーダとサブリーダが決まっている。
PLMを使ったグローバルリアルタイムコラボレーション会議を実施。
5M (人・道具・原料・方法・測定・環境)の基準で変化点を定義し、最新の状態を関係者で共有している
5M変動時の作業指導、品質確認等ができている
製造部門だけでなく開発・設計などホワイトカラー業務にも適用できれいる。
モジュール化の定義、進め方が決められており、原価まで繋がった評価ができる
コンフィグレータが、製品構成、評価、生産まで考慮した仕様になっている。
KGI/KPIに標準化の指標がある
製品開発ゲートの定義と、フェーズごとのアプトプットが明確になっている
先行開発ゲートの定義は、通常開発の定義と異なっており、技術の棚という考え方がある
過去の作業マニュアルが適切に更新され、処分されている
製品・部品の品質管理が定常的に実施されており、管理図などによりモニタリングしている
デザインレビューのパフォーマンスを測定できる指標がある。
グローバルに関連部門が一元化した製品情報を使って実施できる。
DR結果を一元化したE-BOMに蓄積して、再利用できる。
施策は、複数の候補を用意し、各効果を定量的に測れる、
工程設計をの５点満点評価スキルマップを作成し、育成計画との連動させる。
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
チェックリスト（発生防止）とシート（流出）を使い分けている。
CE、リーダからメンバーまで全員で確認できるしくみがある。
目的、仕様確認をスムーズにするためにカテゴリー化できている。
原価情報（試作・量産別）の目標と現状が確認でき

In [29]:
test(10 )

##########部品##########
エンジン搭載 燃料ポンプ　絶縁体
##########対策##########
多能工化を設計者に適用。メカ・電気・ソフトの各スキルマップを作成し統合。
定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。
６ヶ月に一回のサイクルで実施。
会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している
背景・目的・目標・KGI/KPI・大日程、現状の課題までが繋がっている
関連部門が参加した課題解決の仕組みがあり、PLMを使ったデジタル化によるグローバル会議ができる
機能と役割分担表があり、リーダとサブリーダが決まっている。
PLMを使ったグローバルリアルタイムコラボレーション会議を実施。
5M (人・道具・原料・方法・測定・環境)の基準で変化点を定義し、最新の状態を関係者で共有している
5M変動時の作業指導、品質確認等ができている
製造部門だけでなく開発・設計などホワイトカラー業務にも適用できれいる。
モジュール化の定義、進め方が決められており、原価まで繋がった評価ができる
コンフィグレータが、製品構成、評価、生産まで考慮した仕様になっている。
KGI/KPIに標準化の指標がある
製品開発ゲートの定義と、フェーズごとのアプトプットが明確になっている
先行開発ゲートの定義は、通常開発の定義と異なっており、技術の棚という考え方がある
過去の作業マニュアルが適切に更新され、処分されている
製品・部品の品質管理が定常的に実施されており、管理図などによりモニタリングしている
デザインレビューのパフォーマンスを測定できる指標がある。
グローバルに関連部門が一元化した製品情報を使って実施できる。
DR結果を一元化したE-BOMに蓄積して、再利用できる。
施策は、複数の候補を用意し、各効果を定量的に測れる、
工程設計をの５点満点評価スキルマップを作成し、育成計画との連動させる。
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
チェックリスト（発生防止）とシート（流出）を使い分けている。
CE、リーダからメンバーまで全員で確認できるしくみがある。
目的、仕様確認をスムーズにするためにカテゴリー化できている。
原価情報（試作・量産別）の目標と現状が確認できる
過去の

In [30]:
test(50)

##########部品##########
エンジン本体 オイルパン　サブアセンブリ
##########対策##########
前機種の振り返りを実施。目標に対する達成度、課題と次の一手を適用している
QCDの視点で、目標を設定。関連部品にも目標展開、仕様を決めた後、全体目標をアラインしている。
目標を元にKGI/KPIを設定している。
原価企画が全社的に認知されており、全社損益とリンク
原価目標の設定は原価企画を適用し、施策と削減余地までを見積る。
部品別、ライン別、事業別に原価を把握
社内コストテーブルがあり、関連部門と連携しながら原価算出
原価企画部門があり、システムも構築
絶対値・差額見積りの両方を実施
プロジェクト毎の開発費が設定され、目標管理されている
設備投資企画を実施、全社損益とリンクしている
設備投資企画のスケジュールがあり、投資（償却費）と工数の相関を確認
予算管理が全社損益とリンク
ベンチマーキングを実施、コストDBに蓄積
VE/VAを実施。低減のためのアイデアの棚がある
量産開始安定後の実績原価を把握
原価を維持・改善する活動を実施
個別原価が会社損益とリンク
調達に関し管理部門とライン部門がある
調達の必要工数につき全て査定基準がある
工法（プレス、樹脂等）のコストテーブルがある
現地会社の現調率が各国の規制値をクリヤー
メンテナンスコストやクレーム対応コストを加味した原価設計、歩留まり率設計を実施している
法規要求、コラム、ペダルストロークなどの基準を満足するシートHP決定する
CAE解析データを作成し衝突ストーリーを作成する
8の字旋回実働ひずみ測定
ベンチεーM線図測定
ベンチM－N線図測定
ハブボルトスリップトルク測定
Ｌスクリーニング(耐久テスト)
台上耐久
塩害テスト
着氷テスト
実車異音評価
車両組付性
車両、環境、周辺部品の変化点、変更点を確認し実施有無判断
要求管理・製品コンフィグレータをCADと連携している
システムズ・エンジニアリングを導入してモデルベース開発を実施
モデルベース開発を実施、１Dのシミュレーションで機能評価を行っている
E-BOMに原価情報を入れ、モデルベース開発を使ったVEを実施
関連技術情報をCADとマクロで連携させている
AI/機械学習と製品３Dデータを連携し自動設計を実現している
解析と３

In [31]:
test(100)

##########部品##########
エンジン本体 クリーナーアセンブリ、エアW /エレメント
##########対策##########
多能工化を設計者に適用。メカ・電気・ソフトの各スキルマップを作成し統合。
定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。
６ヶ月に一回のサイクルで実施。
会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している
背景・目的・目標・KGI/KPI・大日程、現状の課題までが繋がっている
関連部門が参加した課題解決の仕組みがあり、PLMを使ったデジタル化によるグローバル会議ができる
機能と役割分担表があり、リーダとサブリーダが決まっている。
PLMを使ったグローバルリアルタイムコラボレーション会議を実施。
5M (人・道具・原料・方法・測定・環境)の基準で変化点を定義し、最新の状態を関係者で共有している
5M変動時の作業指導、品質確認等ができている
製造部門だけでなく開発・設計などホワイトカラー業務にも適用できれいる。
モジュール化の定義、進め方が決められており、原価まで繋がった評価ができる
コンフィグレータが、製品構成、評価、生産まで考慮した仕様になっている。
KGI/KPIに標準化の指標がある
製品開発ゲートの定義と、フェーズごとのアプトプットが明確になっている
先行開発ゲートの定義は、通常開発の定義と異なっており、技術の棚という考え方がある
過去の作業マニュアルが適切に更新され、処分されている
製品・部品の品質管理が定常的に実施されており、管理図などによりモニタリングしている
デザインレビューのパフォーマンスを測定できる指標がある。
グローバルに関連部門が一元化した製品情報を使って実施できる。
DR結果を一元化したE-BOMに蓄積して、再利用できる。
施策は、複数の候補を用意し、各効果を定量的に測れる、
工程設計をの５点満点評価スキルマップを作成し、育成計画との連動させる。
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
チェックリスト（発生防止）とシート（流出）を使い分けている。
CE、リーダからメンバーまで全員で確認できるしくみがある。
目的、仕様確認をスムーズにするためにカテゴリー化できている。
原価情報（試作・量産別）の目標と

In [32]:
test(150)

##########部品##########
エンジン本体 持ち分
##########対策##########
多能工化を設計者に適用。メカ・電気・ソフトの各スキルマップを作成し統合。
定性的なアウトプットである個人の振り返りを実施。個人の成長・マネジメント軸を強化する。
６ヶ月に一回のサイクルで実施。
会社目標から製品、各機能組織ごとの、目標展開を実施。メンバー目標とも連携している
背景・目的・目標・KGI/KPI・大日程、現状の課題までが繋がっている
関連部門が参加した課題解決の仕組みがあり、PLMを使ったデジタル化によるグローバル会議ができる
機能と役割分担表があり、リーダとサブリーダが決まっている。
PLMを使ったグローバルリアルタイムコラボレーション会議を実施。
5M (人・道具・原料・方法・測定・環境)の基準で変化点を定義し、最新の状態を関係者で共有している
5M変動時の作業指導、品質確認等ができている
製造部門だけでなく開発・設計などホワイトカラー業務にも適用できれいる。
モジュール化の定義、進め方が決められており、原価まで繋がった評価ができる
コンフィグレータが、製品構成、評価、生産まで考慮した仕様になっている。
KGI/KPIに標準化の指標がある
製品開発ゲートの定義と、フェーズごとのアプトプットが明確になっている
先行開発ゲートの定義は、通常開発の定義と異なっており、技術の棚という考え方がある
過去の作業マニュアルが適切に更新され、処分されている
製品・部品の品質管理が定常的に実施されており、管理図などによりモニタリングしている
デザインレビューのパフォーマンスを測定できる指標がある。
グローバルに関連部門が一元化した製品情報を使って実施できる。
DR結果を一元化したE-BOMに蓄積して、再利用できる。
施策は、複数の候補を用意し、各効果を定量的に測れる、
工程設計をの５点満点評価スキルマップを作成し、育成計画との連動させる。
職務拡充により作業者のモチベーション向上。品質を作りこむ意識が醸成。
チェックリスト（発生防止）とシート（流出）を使い分けている。
CE、リーダからメンバーまで全員で確認できるしくみがある。
目的、仕様確認をスムーズにするためにカテゴリー化できている。
原価情報（試作・量産別）の目標と現状が確認できる
過去の不具合に対し